In [ ]:
from qibo.models import Circuit
from qibo import gates
import numpy as np
from tqdm.auto import tqdm
import lmfit
import matplotlib.pyplot as plt

import qililab as ql
from qililab.circuit_transpiler.native_gates import Wait
from qililab.calibration.calibration_node import export_nb_outputs


ql.logger.setLevel(40)

In [ ]:
qubit = 0
M_BUFFER_TIME = 0
HW_AVG = 3000
REPETITION_DURATION = 200_000
check = False
number_of_random_datapoints = 10
compare_fit = []
PLATFORM_PATH = "../../runcards/example_runcard.yml"
partition = "example_partition"

### Define sequence

In [ ]:
circuit_settings = {
    "circuits": [
        {"name": "II", "gates": ["I", "I", "M"], "params": [-1, -1, -1]},
        {"name": "XpXp", "gates": ["RX", "RX", "M"], "params": [1, 1, -1]},
        {"name": "YpYp", "gates": ["RY", "RY", "M"], "params": [1, 1, -1]},
        {"name": "XpYp", "gates": ["RX", "RY", "M"], "params": [1, 1, -1]},
        {"name": "YpXp", "gates": ["RY", "RX", "M"], "params": [1, 1, -1]},
        {"name": "X9I", "gates": ["RX", "I", "M"], "params": [0.5, -1, -1]},
        {"name": "Y9I", "gates": ["RY", "I", "M"], "params": [0.5, -1, -1]},
        {"name": "X9Y9", "gates": ["RX", "RY", "M"], "params": [0.5, 0.5, -1]},
        {"name": "Y9X9", "gates": ["RY", "RX", "M"], "params": [0.5, 0.5, -1]},
        {"name": "X9Yp", "gates": ["RX", "RY", "M"], "params": [0.5, 1, -1]},
        {"name": "Y9Xp", "gates": ["RY", "RX", "M"], "params": [0.5, 1, -1]},
        {"name": "XpY9", "gates": ["RX", "RY", "M"], "params": [1, 0.5, -1]},
        {"name": "YpX9", "gates": ["RY", "RX", "M"], "params": [1, 0.5, -1]},
        {"name": "X9Xp", "gates": ["RX", "RX", "M"], "params": [0.5, 1, -1]},
        {"name": "XpX9", "gates": ["RX", "RX", "M"], "params": [1, 0.5, -1]},
        {"name": "Y9Yp", "gates": ["RY", "RY", "M"], "params": [0.5, 1, -1]},
        {"name": "YpY9", "gates": ["RY", "RY", "M"], "params": [1, 0.5, -1]},
        {"name": "XpI", "gates": ["RX", "I", "M"], "params": [1, -1, -1]},
        {"name": "YpI", "gates": ["RY", "I", "M"], "params": [1, -1, -1]},
        {"name": "X9X9", "gates": ["RX", "RX", "M"], "params": [0.5, 0.5, -1]},
        {"name": "Y9Y9", "gates": ["RY", "RY", "M"], "params": [0.5, 0.5, -1]},
    ]
}

circuits = []
for circ_settings in circuit_settings["circuits"]:
    circuit = Circuit(qubit + 1)
    c_gates = circ_settings["gates"]
    gates_params = circ_settings["params"]
    for gate, gate_param in zip(c_gates, gates_params):
        if gate == "RX":
            circuit.add(ql.Drag(q=qubit, theta=gate_param * np.pi, phase=0))
        elif gate == "RY":
            circuit.add(ql.Drag(q=qubit, theta=gate_param * np.pi, phase=np.pi / 2))
        elif gate == "I":
            circuit.add(ql.Drag(q=qubit, theta=0, phase=0))
    circuit.add(gates.M(qubit))
    circuits.append(circuit)

### Platform initialization

In [ ]:
platform = ql.build_platform(runcard=PLATFORM_PATH)

### Slurm job name

In [ ]:
job_name = f"AC-xy-q{qubit}"

### Run sequence

In [ ]:
%%submit_job -o results -p $partition -n $job_name

platform.connect()
platform.initial_setup()
platform.turn_on_instruments()
results = []

for circuit in tqdm(circuits, total=len(circuits)):
    result = platform.execute(program=circuit, num_avg=HW_AVG, repetition_duration=REPETITION_DURATION)
    results.append(result.array)

platform.disconnect()

In [ ]:
results = results.result()
loops = {}
ql.save_results(results=results, loops=loops, data_path="../data/", name=job_name)
i, q = np.hstack(results)

fig, axs = plt.subplots(1, 2, figsize=(13, 7))
axs[0].plot(i, "--o", color="blue")
axs[1].plot(q, "--o", color="blue")
axs[0].set_title("I")
axs[1].set_title("Q")
axs[0].set_ylabel("Voltage [a.u.]")
axs[1].set_ylabel("Voltage [a.u.]")
ax = fig.gca()
fig.suptitle(f"Allxy q{qubit}")
fig.tight_layout()

### Define plotting functions

In [ ]:
def plot_iq(xdata, results: np.ndarray, title_label: str, xlabel: str, check=False):
    fig, axes = plt.subplots(1, 2, figsize=(13, 7))
    if check:
        axes[0].scatter(xdata, results[0])
        axes[1].scatter(xdata, results[1])
    else:
        axes[0].plot(xdata, results[0], "--o", color="blue")
        axes[1].plot(xdata, results[1], "--o", color="blue")
    axy = axes[1].twiny()
    # other_xlim = axes[1].get_xlim()
    axy.set_xlim(-10, 10)
    axes[0].set_title("I")
    axes[1].set_title("Q")
    axes[0].set_xlabel(xlabel)
    axes[1].set_xlabel(xlabel)
    axes[0].set_ylabel("Voltage [a.u.]")
    axes[1].set_ylabel("Voltage [a.u.]")
    fig.suptitle(title_label)
    return fig, axes

### Fit data

In [ ]:
# Initialize dictionaries
r_squared = {"i": 0, "q": 0}
fitted_ifs = {"i": None, "q": None}
fitted_eval_funcs = []

# Loop over quadratures
for fit_quadrature in ["i", "q"]:
    # Fitting function
    def lorentzian(x, amplitude, center, width, offset):
        return amplitude / (1 + ((x - center) / (0.5 * width)) ** 2) + offset

    # Fit signal
    fit_signal = results[0] if fit_quadrature == "i" else results[1]

    # Lorentzian fit
    mod = lmfit.Model(lorentzian)

    # Set initial parameter values
    initial_amp = max(fit_signal) - min(fit_signal)
    if (np.mean(fit_signal) - min(fit_signal)) ** 2 > (np.mean(fit_signal) - max(fit_signal)) ** 2:
        # if the mean of the values is closer to the maximum, it means we have negative amplitude!
        initial_amp = -initial_amp

    mod.set_param_hint(
        "amplitude",
        value=initial_amp,
        min=-abs(max(fit_signal) - min(fit_signal)),
        max=abs(max(fit_signal) - min(fit_signal)),
    )
    mod.set_param_hint("center", value=np.mean(sweep_interval), min=np.min(sweep_interval), max=np.max(sweep_interval))
    mod.set_param_hint(
        "width",
        value=(max(sweep_interval) - min(sweep_interval)) / 10,
        min=(max(sweep_interval) - min(sweep_interval)) / 100,
        max=(max(sweep_interval) - min(sweep_interval)),
    )
    mod.set_param_hint("offset", value=np.mean(fit_signal), min=np.min(fit_signal), max=np.max(fit_signal))

    params = mod.make_params()
    fit = mod.fit(data=fit_signal, params=params, x=sweep_interval, method="differential_evolution")

    # Update r_squared value
    r_squared[fit_quadrature] = fit.rsquared

    fitted_amplitude = fit.params["amplitude"].value
    fitted_center = fit.params["center"].value
    fitted_width = fit.params["width"].value
    fitted_offset = fit.params["offset"].value

    popt = [fitted_amplitude, fitted_center, fitted_width, fitted_offset]

    xdata = np.linspace(min(sweep_interval), max(sweep_interval), num=1000)

    if fitted_amplitude < 0:
        fitted_if = sweep_interval[
            np.argmin(lorentzian(sweep_interval, fitted_amplitude, fitted_center, fitted_width, fitted_offset))
        ]
    else:
        fitted_if = sweep_interval[
            np.argmax(lorentzian(sweep_interval, fitted_amplitude, fitted_center, fitted_width, fitted_offset))
        ]

    fitted_ifs[fit_quadrature] = fitted_if
    fitted_func = lorentzian(sweep_interval, *popt)
    fitted_eval_funcs.append(fitted_func)

    # Plot
    fig, axes = plot_iq(
        xdata=sweep_interval, results=results, title_label=f"2 Tone q{qubit}", xlabel="Amplitude", check=check
    )
    ax = axes[0 if fit_quadrature == "i" else 1]
    if check and len(compare_fit) > 0:
        fitted_func = compare_fit[1][0] if fit_quadrature == "i" else compare_fit[1][1]
        ax.plot(compare_fit[0], fitted_func, "--", label="previously calibrated fit", color="red")
        ax.legend()
    else:
        label_fit = f"FITTED IF = {fitted_if}"
        ax.plot(sweep_interval, fitted_func, "--", label=label_fit, color="red")
    ax.legend()

# Display results or further analysis as needed

### Select best fit and update runcard

In [ ]:
best_quadrature = "i" if r_squared["i"] > r_squared["q"] else "q"
best_fit = fitted_ifs[best_quadrature]

print(f"Fitted IF: {best_fit}")

In [ ]:
key = "IF"
print(f"OUTPUTS{key}:{best_fit}")

In [ ]:
platform_parameters = [(ql.Parameter.IF, float(best_fit), f"drive_q{qubit}_bus", None)]

In [ ]:
export_nb_outputs(
    {
        "check_parameters": {"sweep_interval": np.arange(1, 20, 1).tolist(), "results": [i.tolist(), q.tolist()]},
    }
)